### 通用 Object 格式

每个 Object 可以抽象成一个 List，List 里面的每一项对应一个刚性部件。每个刚性部件可以用一个字典来描述，字典的格式如下（*代表非必须，依使用的功能而定）：

```json
{
    "points": n*3 np.ndarray,
    "joint_data_origin": [x0, y0, z0],
    "joint_data_direction": [x1, y1, z1],
    "limit": [p_min, p_max, r_min, r_max],
    "dfn": dfs number,
    "dfn_fa": father's dfs number,
    *"shape_code": latent code for genSDF,
    *"bbox_l": [l_x, l_y, l_z],
    *"bbox_center": [x_c, y_c, z_c],
    *"rho": number of point per unit cube (1x1x1),
    *"mesh": triMesh object,
}
```

将该 List 直接使用 pickle 序列化为 .dat 格式，即可作为 ID 计算脚本的输入：

```python
obj = # List of PartDict
path = "<path_to_directory>" / "<name_of_object>.dat"
with open(path, 'wb') as f: 
    f.write(pickle.dumps(obj))
```

### Compute Metrics 参数设置

该计算脚本基于运行环境：`conda activate pytorch3d`，可以通过 NAP 根目录下 metric_env.sh 构建。

请在下面的代码块中填入正确的路径或文件名。

In [1]:
# data_dir: same as '--data_dir' in instantiation_distance.py
data_dir = "/root/workspace/csn4bls7v38s73cem970/6_ours_obj_dats"
# data_dir = "testin"

# output_name: directory name of '--data_dir' in instantiation_distance.py 
# gt_name: directory name of '--gt_dir' in instantiation_distance.py
output_name = data_dir.split('/')[-1]
gt_name = "data_gt"
# gt_name = "testgt"

# N_states: same as '--N_states' in instantiation_distance.py
# N_pcl: same as '--N_pcl' in instantiation_distance.py
# n_sample_POR: number of sample points in the Part Overlapport Ratio (POR) calculation, recommended to be not less than 4096
N_states = 10
N_pcl = 4096
n_sample_POR = 4096

# result_dir: same as '--output_dir' in instantiation_distance.py
result_dir = '/root/workspace/csn4bls7v38s73cem970/output/ours'

# sample_file_path: same as '--sample_file_path' in instantiation_distance.py
sample_file_path = '/root/workspace/csn4bls7v38s73cem970/eval/selected_files.json'

##########
print("data_dir: ", data_dir)
print("output_name: ", output_name)
print("gt_name: ", gt_name)
print("N_states: ", N_states)
print("N_pcl: ", N_pcl)
print("result_dir: ", result_dir)

data_dir:  /root/workspace/csn4bls7v38s73cem970/6_ours_obj_dats
output_name:  6_ours_obj_dats
gt_name:  data_gt
N_states:  10
N_pcl:  4096
result_dir:  /root/workspace/csn4bls7v38s73cem970/output/ours


### ID Metrics 计算

需要准备好 model output 和 gt files，分别放在两个文件夹下，文件夹里面就是.dat格式保存的obj dict文件，每个文件代表一个物品。
参照 `compute_id.sh` 里的格式，填好参数并运行 `instantiation_distance.py`。

之后把两个参数中的文件夹名，和计算脚本输出的目录填在下面，运行该代码块即可。

### ID Metrics 解释

- minimum matching distance (MMD) 对于每个生成样本，在源数据中找与它距离最小的作为match（minimum matching）。每个生成样本只统计和match的距离，对所有距离取平均得到这一指标。**描述个体维度的重建质量，数值越小越好。**
- coverage (COV) 计算所有match的去重数量，除以输入样本的总数。**描述模型覆盖率，数值越大越好**。
- 1-nearest neighbor accuracy (1-NNA) 直观上说，该指标的数值等于错误匹配的样本数量。**描述分布的相似度，数值越小越好。**

1-NNA的计算过程：把输入样本和生成样本合并为一个大集合，对其进行最邻近聚类（1-NN Clustering）。记输入样本为1，生成样本为0，得到gt。再通过最邻近聚类重新预测标签，即把每个样本预测为它1-NN的gt类别，计算预测准确率，得到该值。

In [2]:
from utils import eval_ID

# evaluate the instantiation distance
eval_ID(result_dir, output_name, gt_name, N_states, N_pcl)

/opt/conda/envs/pytorch3d/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


6_ours_obj_dats data_gt

{'1-NN-ID-acc': tensor(0.5700),
 'lgam_cov-ID': tensor(0.4800),
 'lgan_mmd-ID': tensor(0.0287)}


### POR Metric 计算



In [3]:
import os, os.path as osp
import pickle, json
import numpy as np
from tqdm import tqdm
from utils import POR, sample_object, align_part_keys

POR_max_list, POR_mean_list = [], []
in_fn_list = sorted([f for f in os.listdir(data_dir) if f.endswith(".dat")])

if sample_file_path != None:
    with open(sample_file_path, 'r') as f:
        sample_file = json.load(f)
    sample_object(in_fn_list, sample_file)

N_in = len(in_fn_list)

DATA_IN = []
print("caching INPUT ...")
for i in tqdm(range(N_in)):
    fn = osp.join(data_dir, in_fn_list[i])
    data = pickle.load(open(fn, "rb"))
    align_part_keys(data)
    DATA_IN.append(data)

for obj in DATA_IN:
    POR_mean, POR_max = POR(obj, n_sample=n_sample_POR)
    POR_max_list.append(POR_max)
    POR_mean_list.append(POR_mean)

# cache the POR results
pickle.dump(POR_max_list, open(osp.join(result_dir, 'POR_max_list.pkl'), 'wb'))
pickle.dump(POR_mean_list, open(osp.join(result_dir, 'POR_mean_list.pkl'), 'wb'))

result = {
    "Max POR": np.mean(POR_max_list),
    "Mean POR": np.mean(POR_mean_list)
}

print(result)

caching INPUT ...


Processing on different pose state.: 100%|██████████| 10/10 [00:00<00:00, 10.95it/s]


{'Max POR': 0.0189803, 'Mean POR': 0.0061958306}
